In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json 
from pandas.io.json import json_normalize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
import re
from array import array
import string

Using TensorFlow backend.
/Users/melanie/anaconda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def load_book(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [4]:
dracula = load_book('./horrorbooks/dracula.txt')
drjekyll = load_book('./horrorbooks/drjekyllmrhyde.txt')
frankenstein = load_book('./horrorbooks/frankenstein.txt')
theturnofthescrew = load_book('./horrorbooks/theturnofthescrew.txt')
thefallofusher = load_book('./horrorbooks/thefallofusher.txt')
books = dracula + drjekyll + frankenstein + theturnofthescrew
print(books[-100:])

 was
that I held. We were alone with the quiet day, and his little heart,
dispossessed, had stopped.


In [7]:
def preclean(text):
    text = re.sub(r'_(.*)_', '', text)
    text = text.replace('--', '')
    tokens = text.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [8]:
tokenized_books = preclean(books)
print(dracula[:100])

CHAPTER I

JONATHAN HARKER'S JOURNAL

(_Kept in shorthand._)


_3 May. Bistritz._--Left Munich at 8:


In [11]:
# tokenized_dracula = nltk.word_tokenize(dracula)
# lowercase
# tokenized_dracula = [t.lower() for t in tokenized_dracula]
print(len(tokenized_books))

296516
24556


In [12]:
tokens = tokenized_books

length = 50
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
# for i in range(len(tokens)-length):
#     sequence = tokens[i:i+length]
#     line = ' '.join(sequence)
#     sequences.append(line)
print('Total Sequences: %d' % len(sequences))
print(len(sequences[0]))
print(len(sequences[10]))
# def save_doc(lines, filename):
#     data = '\n'.join(lines)
#     file = open(filename, 'w')
#     file.write(data)
#     file.close()
# out_filename = 'dracula_sequences.txt'
# save_doc(sequences, out_filename)

Total Sequences: 296466
260
255


In [13]:
# def load_doc(filename):
#     file = open(filename, 'r')
#     text = file.read()
#     file.close()
#     return text
 
# load
# in_filename = 'dracula_sequences.txt'
# doc = load_doc(in_filename)
# lines = doc.split('\n')

In [14]:
lines = sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

In [15]:
sequences = tokenizer.texts_to_sequences(lines)

In [16]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

15067


In [17]:
# sequences1 = [np.array(s) for s in sequences]
# sequences2 = np.array(sequences1)
print(len(sequences[100]))
sequences1 = np.array([np.array(s) for s in sequences])
X, y = sequences1[:,:-1], sequences1[:,-1]
y = np_utils.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

50


In [18]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 49, 50)            753350    
_________________________________________________________________
lstm_1 (LSTM)                (None, 49, 100)           60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 15067)             1521767   
Total para

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='loss', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
# fit model
model.fit(X, y, batch_size=128, epochs=10, callbacks=[earlystop])

Epoch 1/10
  7936/296466 [..............................] - ETA: 1174s - loss: 7.7557 - acc: 0.0426

KeyboardInterrupt: 

In [240]:
import pickle
# save the model to file
model.save('model.h5')
# save the tokenizer
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [242]:
import random
seed_text = lines[random.randint(0,len(lines))]
print(seed_text + '\n')
encoded_seed_text = tokenizer.texts_to_sequences([seed_text])[0]

we the more unpunctual are the trains what ought they to be in china all day long we seemed to



In [248]:
from keras.preprocessing.sequence import pad_sequences
def generate_sequence(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
    # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [249]:
generate_sequence(model, tokenizer, seq_length, seed_text, 20)

'dawdle through a country which was full of beauty of every kind sometimes we saw little towns or castles on'